In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import gzip
import csv

# Define Input and Output Paths
sts_dataset_path = 'input_data/stsbenchmark.tsv.gz'
model_save_path = 'output_model/SemEval_STSModel_CrossEncoder'

# Use DistillBert Cross Encoder
model = CrossEncoder('distilroberta-base', num_labels=1)

# Read Input Data
train_samples = []
dev_samples = []
test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        if row['split'] == 'dev':
            dev_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
        elif row['split'] == 'test':
            test_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
        else:
            # Pass both combinations to the train set to get symmetric scores, i.e. CrossEncoder(A,B) = CrossEncoder(B,A)
            train_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
            train_samples.append(InputExample(texts=[row['sentence2'], row['sentence1']], label=score))

# Pytorch Dataloader
train_batch_size = 16
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# Training Parameters
num_epochs = 4
warmup_steps = round(len(train_dataloader) * num_epochs * 0.1)

# Define Performance Evaluator
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='valid')

# Train the model
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

##### Load model and eval on test set
#model = CrossEncoder(model_save_path)

# Evaluate Performance Metric
evaluator = CECorrelationEvaluator.from_input_examples(test_samples, name='test')
evaluator(model)